# Register model and deploy as webservice in ACI
 - Register a model in your Azure Machine Learning Workspace.
 - Deploy model as a web service in an Azure Container Instance.

## Prerequisites

Install the Azure Machine Learning Python SDK and create a workspace.

In [1]:
import azureml.core

print('SDK version:', azureml.core.VERSION)

SDK version: 1.18.0


## Initialize workspace

Create a [Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py) object from your persisted configuration.

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

automl
automl
canadacentral
18a48e4d-4b42-45c1-b041-9bcb480c7633


## Load Dataset

In [3]:
from sklearn.datasets import load_diabetes
dataset_x, dataset_y = load_diabetes(return_X_y=True)

In [4]:
import joblib

from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
#from tpot import TPOTRegressor

# model = TPOTRegressor(generations=5, population_size=50, verbosity=3, random_state=42)
# model.fit(dataset_x, dataset_y)

model = Ridge().fit(dataset_x, dataset_y)

joblib.dump(model, 'sklearn_regression_model.pkl')

['sklearn_regression_model.pkl']

In [5]:
dataset_x[0:3,]
dataset_x.shape

(442, 10)

## Register model

Register a model by calling [Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none-).

In [6]:
import sklearn
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration


model = Model.register(workspace=ws,
                       model_name='my-sklearn-model',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

Registering model my-sklearn-model
Name: my-sklearn-model
Version: 4


## Deploy model

Deploy your model as a web service using [Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-). Deploy your scikit-learn model to an Azure Container Instance (ACI).

In [7]:
service_name = 'my-sklearn-service'

service = Model.deploy(ws, service_name, [model], overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [8]:
service.scoring_uri

'http://d42b2087-f92c-47d1-98e6-da46051f6b6f.canadacentral.azurecontainer.io/score'

After your model is deployed, perform a call to the web service using [service.run()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#run-input-).

In [ ]:
import json

input_payload = json.dumps({
    'data': dataset_x[0:2].tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

When you are finished testing your service, clean up the deployment with [service.delete()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#delete--).

In [ ]:
service.delete()

### Use a custom environment
Specify the model's runtime environment by creating an [Environment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py) object and providing the [CondaDependencies](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) needed by your model.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [ ]:
with open('score.py') as f:
    print(f.read())

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'my-custom-env-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

After your model is deployed, make a call to the web service using [service.run()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#run-input-).

In [ ]:
input_payload = json.dumps({
    'data': dataset_x[0:2].tolist()
})

output = service.run(input_payload)

print(output)

When you are finished testing your service, clean up the deployment with [service.delete()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#delete--).

In [ ]:
service.delete()

## Next steps

 - To run a production-ready web service, see the [notebook on deployment to Azure Kubernetes Service](../production-deploy-to-aks/production-deploy-to-aks.ipynb).